In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal
import scipy.stats

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""


def shift_and_sum(signal, x_range, y_range, N):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    x_antenna = x_antenna.reshape((1, x_antenna.size)).repeat(N ** 2, axis = 0)
    y_antenna = y_antenna.reshape((1, y_antenna.size)).repeat(N ** 2, axis = 0)
    
    x_array = np.linspace(-x_range, x_range, N)
    y_array = np.linspace(-y_range, y_range, N)
    
    x_grid, y_grid = np.meshgrid(x_array, y_array)
    x_grid = x_grid.flatten().reshape((x_grid.size, 1)).repeat(nch, axis=-1)
    y_grid = y_grid.flatten().reshape((y_grid.size, 1)).repeat(nch, axis=-1)
    
    #print(x_antenna.shape, x_grid.shape)
    
    phase_grid = 2 * np.pi * np.sqrt((x_grid - x_antenna) ** 2 + (y_grid - y_antenna) ** 2) / wavelength_lo
    
    anti_spiral = angles.reshape((1, angles.size)).repeat(N**2, axis=0)
    
    summed_signals = np.matmul(np.exp(1j * (phase_grid-anti_spiral)), signal)
    
    summed_signals = summed_signals.reshape((N, N, summed_signals.shape[-1]))
    
    return summed_signals


def shift_signal(signal, x_new, y_new):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    
    
    phase_shift = 2 * np.pi * (d_new - d_old) / wavelength_lo
    #print(phase_shift)
    
    shifted_signal = np.exp(-1j * phase_shift) * signal
    
    return shifted_signal

def shift_pc(pc, x_new, y_new):
    
    nch = 60
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    
    shifted_pc = np.zeros(nch * pc.size, np.complex64)
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)

    d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    

    phase_shift = 2 * np.pi * (d_new - d_old) / wavelength_lo + angles
    
    for i, phase in enumerate(phase_shift):
        shifted_pc[i * pc.size:(i + 1) * pc.size] = np.exp(-1j * phase) * pc
    

    return shifted_pc

def tile_pc(pc):
    
    nch = 60
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    #r_array = 0.10
    #wavelength_lo = 3e8 / 25.86e9
    
    tile_pc = np.zeros(nch * pc.size, np.complex64)
    
    phase_shift = angles
    
    for i, phase in enumerate(phase_shift):
        tile_pc[i * pc.size:(i + 1) * pc.size] = np.exp(-1j * phase) * pc
    

    return tile_pc

def track_length_weights(rng, nslice, mean=3, invert=False):
    est_track_length = rng.exponential(mean)
    
    #print(est_track_length, np.ceil(est_track_length), int(nslice - np.ceil(est_track_length)),1 - np.ceil(est_track_length) + est_track_length, nslice)
    
    slice_weights = np.ones(nslice)
    
    if est_track_length >= nslice:
        return slice_weights
    else:
        N_empty_slice = int(nslice - np.ceil(est_track_length))
        
        partial_slice_weight = 1 - np.ceil(est_track_length) + est_track_length
        
        for n in range(N_empty_slice):
            slice_weights[n] = 0.0
            
        slice_weights[N_empty_slice] = partial_slice_weight
        
        if rng.integers(0, 2) == 1 and invert:
            slice_weights = np.flip(slice_weights)
        
        return slice_weights

    
def add_noise(signal, var):
    
    shape = signal.shape
    rng = np.random.default_rng()
    
    noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, shape[0] * shape[1])
    noise = np.fft.fft((noise[:, 0] + 1j * noise[:, 1]).reshape(shape), axis=-1) / 8192
    
    return signal + noise
    
var = 1.38e-23 * 10 * 50 * 200e6

In [ ]:
os.listdir(os.path.join(PATH, 'damselfly/data'))

In [ ]:
os.listdir(os.path.join(PATH, 'damselfly/data/sim_data'))

In [ ]:
os.listdir(os.path.join(PATH, 'damselfly/data/sim_data'))

# load h5file of data to augment

In [ ]:
#data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
#data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '210729_df_84_1slice.h5')

#h5datafile = h5py.File(data, 'r')

#energies = h5datafile['meta']['energy'][:]

#select_energies = np.argwhere(abs(energies - 18550) <= 0.5).squeeze()

In [ ]:
#data = os.path.join(os.path.join(PATH, 'mayfly/data/datasets'), '211002_mf_84_100_slice8192.h5')
data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '211019_84_1d2sl_nosum_fft.h5')
h5val_signals = h5py.File(data, 'r')

data = os.path.join(os.path.join(PATH, 'damselfly/data/sim_data'), '211019_84_100_1d2sl_nosum_fft.h5')
h5train_signals = h5py.File(data, 'r')

#print(h5val_signals['signal']['0'].shape)
Nval_signal = len(list(h5val_signals['signal'].keys()))
Ntrain_signal = len(list(h5train_signals['signal'].keys()))
#Nsignal = h5datafile['data'].shape[0]

#Nsignal = len(list(h5datafile['signal'].keys()))
Nslice = 2
Nsample = 8192
Nch = 60

#dataset = np.zeros((Nsignal, Nch * Nsample), dtype=np.complex64)
#print(dataset.shape)

#for i in range(Nsignal):
#    dataset[i, :] = np.fft.fftshift(np.fft.fft(h5datafile['data'][i, :].reshape(60, 8192).sum(axis=0))) / 8192
#    dataset[i, :] = h5datafile['signal'][f'{i}'][0, :]
#    dataset[i, :] = (np.fft.fftshift((np.fft.fft(h5datafile['data'][i, :].reshape(60, 8192), axis=-1)), axes=-1) / 8192).flatten()
#    if i % 250 == 249:
#        print(i + 1)

# define dataset

In [ ]:
os.listdir(os.path.join(PATH, 'damselfly/data/datasets'))

In [ ]:

Ncopies_train = 20
noise_frac = 0.2
Nnoise = int(Ncopies_train * Ntrain_signal / (1 / noise_frac - 1))
train_dataset_shape = (Ncopies_train * Ntrain_signal + Nnoise, Nslice, Nch * Nsample)
train_labels = np.ones(train_dataset_shape[0])
train_labels[Ncopies_train * Ntrain_signal:] = 0
train_name = '211020_84_100_1d2sl4mt5cm_nosum_fft_train.h5'

Ncopies_val = 20
noise_frac = 0.5
Nnoise = int(Ncopies_val * Nval_signal / (1 / noise_frac - 1))
val_dataset_shape = (Ncopies_val * Nval_signal + Nnoise, Nslice, Nch * Nsample)
val_labels = np.ones(val_dataset_shape[0])
val_labels[Ncopies_val * Nval_signal:] = 0
val_name = '211020_84_100_1d2sl4mt5cm_nosum_fft_val.h5'
test_name = '211020_84_100_1d2sl4mt5cm_nosum_fft_test.h5'

print(train_dataset_shape, val_dataset_shape)

h5train = h5py.File(os.path.join(PATH, 'damselfly/data/datasets', train_name), 'w')
h5train_data = h5train.create_dataset('data', train_dataset_shape, dtype='c8')
h5train_labels = h5train.create_dataset('label', (train_dataset_shape[0],), dtype='i4')
h5train.close()

h5val = h5py.File(os.path.join(PATH, 'damselfly/data/datasets', val_name), 'w')
h5val_data = h5val.create_dataset('data', val_dataset_shape, dtype='c8')
h5val_labels = h5val.create_dataset('label', (val_dataset_shape[0],), dtype='i4')
h5val.close()

h5test = h5py.File(os.path.join(PATH, 'damselfly/data/datasets', test_name), 'w')
h5test_data = h5test.create_dataset('data', val_dataset_shape, dtype='c8')
h5test_labels = h5test.create_dataset('label', (val_dataset_shape[0],), dtype='i4')
h5test.close()

In [ ]:
h5val.close()

# create training dataset for pca projection

In [ ]:
signal_index = 0

h5train = h5py.File(os.path.join(PATH, 'damselfly/data/datasets', train_name), 'r+')
h5train_data = h5train['data']
h5train_labels = h5train['label']
h5train_labels[:] = train_labels
print(h5train['data'].shape)

rng = np.random.default_rng()
for itrain, key in enumerate(h5train_signals['signal'].keys()):
    signal = h5train_signals['signal'][key][:]
    for ncopy in range(Ncopies_train):
        
        r_shift = rng.random(1,) * 5e-2 # random distance between 0 and 5 cm

        slice_weights = track_length_weights(rng, Nslice, mean=4, invert=True)
        
        for nslice in range(Nslice):
            
            shifted_signal = shift_signal(signal[nslice, :], r_shift, 0.00)
            
            noise = add_noise(np.zeros(shifted_signal.shape), var)
            
            h5train_data[signal_index, nslice, :] = noise.flatten() + shifted_signal.flatten() * slice_weights[nslice]
        
        signal_index += 1
        
        if signal_index % 10000 == 9999:
            print(signal_index + 1)
        
for inoise in range(h5train_data.shape[0] - Ncopies_train * Ntrain_signal):
    
    for nslice in range(Nslice):
        
        noise = add_noise(np.zeros(shifted_signal.shape), var)
    
        h5train_data[signal_index, nslice, :] = noise.flatten()
        
    signal_index += 1
    
    if signal_index % 10000 == 9999:
            print(signal_index + 1)
            
        
        
h5train.close()

In [ ]:
test_signal = h5train_signals['signal']['35'][:]
print(h5train_signals['signal']['35'].attrs['theta_min'])

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

#ax.plot(abs(test_signal[0,:].flatten()))
#ax.set_xlim(0, 8192)

antispiral = np.exp(-1j * np.arange(0, 60, 1) * 2 * np.pi / 60)

summed_signal = (test_signal[0,:].T * antispiral).T.sum(axis=0)

noisy_signal = add_noise(np.zeros((60, 8192)), var)

sum_noisy_signal = (noisy_signal.T * antispiral).T.sum(axis=0)

ax.plot(abs(sum_noisy_signal) ** 2)
ax.plot(abs(summed_signal) ** 2)
#ax.plot(60 * abs(test_signal[0 , :].flatten()))
ax.set_xlim(0, 8192)

In [ ]:
n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :]

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(signal.real)
ax.plot(signal.imag)

tiled_pc = tile_pc(pc)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(tiled_pc.real)
ax.plot(tiled_pc.imag)




# correlation using unsummed signal and repeated pc

In [ ]:

n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :]

tiled_pc = tile_pc(pc)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(tiled_pc))
ax.plot(abs(pc))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(signal))
ax.plot(abs(pc) / 1e8)

In [ ]:

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(circ_conv))
ax.plot(abs(signal))
#ax.set_xlim(0, 5*8192)

# shift signal to x=1cm, correlate with unshifted pc

In [ ]:
n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

tiled_pc = tile_pc(pc)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude goes down')


# shift signal to x=1cm, correlate with shifted pc

In [ ]:
n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

pc = shift_pc(pc, 0.01, 0.0)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude back to previous peak, asymmetric convolution')


# shift signal to y=1cm, correlate with shifted pc to x=1cm

In [ ]:
n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.00, 0.01).flatten()

pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('different angular position just circular shifts the convolution')

# signal auto-correlation

In [ ]:
n = 2
m = 3

#pc = select_pc[n, :]
signal = dataset[m, :]

norm_signal = signal / np.sqrt((abs(signal) ** 2).sum())

#signal = shift_signal(signal, 0.00, 0.01).flatten()

#pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal)) * np.fft.fft(signal)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

# figure out noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(noise))
ax.plot(abs(signal))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(scipy.signal.fftconvolve(noise + signal, signal)))

# signal auto-correlation with noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 3

#pc = select_pc[n, :]
signal = dataset[m, :]


#signal = shift_signal(signal, 0.00, 0.01).flatten()

#pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(scipy.signal.fftconvolve(signal + noise, signal))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

circ_conv = abs(np.fft.ifft(np.fft.fft(signal + noise) * np.fft.fft(signal)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))

# correlation using unsummed signal and repeated pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :]

tiled_pc = tile_pc(pc)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(noise)) * np.fft.fft(tiled_pc)))
ax.plot(abs(circ_conv))

# shift signal to x=1cm, correlate with unshifted pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

tiled_pc = tile_pc(pc)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(tiled_pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude goes down')


# shift signal to x=1cm, correlate with shifted pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.01, 0.0).flatten()

pc = shift_pc(pc, 0.01, 0.0)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('amplitude back to previous peak, asymmetric convolution')


# shift signal to y=1cm, correlate with shifted pc, add noise

In [ ]:
# noise #
var = 1.38e-23 * 10 * 200e6 * 50
noise = np.random.multivariate_normal([0,0], np.eye(2) * var / 2, 60 * 8192)
noise = noise[:, 0] + 1j * noise[:, 1]
noise = (np.fft.fft(noise.reshape(60, 8192), axis=-1) / 8192).flatten()
# noise #

n = 2
m = 3

pc = select_pc[n, :]
signal = dataset[m, :].reshape(60, 8192)

signal = shift_signal(signal, 0.00, 0.01).flatten()

pc = shift_pc(pc, 0.01, 0.00)

#tile_pc = np.tile(pc, 60)

circ_conv = abs(np.fft.ifft(np.fft.fft(np.flip(signal + noise)) * np.fft.fft(pc)))

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(circ_conv))
ax.set_title('different angular position just circular shifts the convolution')